In [6]:
import sys
!{sys.executable} -m pip install elasticsearch==6.8.2
!{sys.executable} -m pip install gensim

     |████████████████████████████████| 24.2 MB 421 kB/s eta 0:00:01
     |████████████████████████████████| 119 kB 1.9 MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-4.2.0-py3-none-any.whl size=109630 sha256=961351e66ca6e76e272bcb798b757cbacd1e83a7979ce29ff431bafcd9ccf351
  Stored in directory: /home/vnkrtv/.cache/pip/wheels/25/88/e3/7cd51a6379cac37213cac47545a27688782752ff66351b953d
Successfully built smart-open


In [2]:
from datetime import datetime
from elasticsearch import Elasticsearch

In [4]:
ELASTIC_URL = '0.0.0.0:9200'

es = Elasticsearch(ELASTIC_URL)
es.info()

{'name': '-orWJ9H',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'wGvvrOBZT12LXKS_LIJATg',
 'version': {'number': '6.5.4',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'd2ef93d',
  'build_date': '2018-12-17T21:17:40.758843Z',
  'build_snapshot': False,
  'lucene_version': '7.5.0',
  'minimum_wire_compatibility_version': '5.6.0',
  'minimum_index_compatibility_version': '5.0.0'},
 'tagline': 'You Know, for Search'}

In [11]:
from gensim.corpora.wikicorpus import WikiCorpus

WIKI_DUMP_PATH = '/home/vnkrtv/Downloads/ruwiki-20210301-pages-articles-multistream.xml.bz2'
wiki = WikiCorpus(WIKI_DUMP_PATH, lemmatize=False, dictionary=False)

In [12]:
next(wiki.get_texts())

['литва',
 'официальное',
 'название',
 'лито',
 'вская',
 'респу',
 'блика',
 'государство',
 'расположенное',
 'северной',
 'части',
 'европы',
 'площадь',
 'км²',
 'протяжённость',
 'севера',
 'на',
 'юг',
 'км',
 'запада',
 'на',
 'восток',
 'км',
 'население',
 'составляет',
 'человек',
 'январь',
 'занимает',
 'место',
 'мире',
 'по',
 'численности',
 'населения',
 'по',
 'территории',
 'имеет',
 'выход',
 'балтийскому',
 'морю',
 'расположена',
 'на',
 'его',
 'восточном',
 'побережье',
 'береговая',
 'линия',
 'составляет',
 'всего',
 'км',
 'наименьший',
 'показатель',
 'среди',
 'государств',
 'балтии',
 'на',
 'севере',
 'граничит',
 'латвией',
 'на',
 'юго',
 'востоке',
 'белоруссией',
 'на',
 'юго',
 'западе',
 'польшей',
 'калининградской',
 'областью',
 'россии',
 'сто',
 'ли',
 'ца',
 'вильнюс',
 'официальный',
 'язык',
 'ли',
 'тов',
 'ский',
 'де',
 'неж',
 'ная',
 'еди',
 'ни',
 'ца',
 'евро',
 'восстановление',
 'независимости',
 'страны',
 'провозглашено',
 'марта'

In [30]:
t9_index = {
  "settings": {
    "index": {
      "analysis": {
        "analyzer": {
          "custom_analyzer": {
            "type": "custom",
            "tokenizer": "standard",
            "filter": [
              "lowercase",
              "custom_edge_ngram"
            ]
          }
        },
        "filter": {
          "custom_edge_ngram": {
            "type": "edge_ngram",
            "min_gram": 2,
            "max_gram": 10
          }
        }
      }
    }
  },
  "mappings": {
    "my_type": {
      "properties": {
        "text": {
          "type": "text",
          "analyzer": "custom_analyzer",
          "search_analyzer": "standard"
        }
      }
    }
  }
}

In [35]:
es.indices.create('test_t9', body=t9_index)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_t9'}

In [46]:
print(json.dumps(es.indices.get_settings('test_t9'), indent=4))

{
    "test_t9": {
        "settings": {
            "index": {
                "number_of_shards": "5",
                "provided_name": "test_t9",
                "creation_date": "1615578363995",
                "analysis": {
                    "filter": {
                        "custom_edge_ngram": {
                            "type": "edge_ngram",
                            "min_gram": "2",
                            "max_gram": "10"
                        }
                    },
                    "analyzer": {
                        "custom_analyzer": {
                            "filter": [
                                "lowercase",
                                "custom_edge_ngram"
                            ],
                            "type": "custom",
                            "tokenizer": "standard"
                        }
                    }
                },
                "number_of_replicas": "1",
                "uuid": "Q5bamA9eSgO5NecZjq4jrA"

In [48]:
from typing import Generator, List

import psycopg2


class PostgresStorage:
    """
    Base class for working with PostgreSQL
    """

    conn: psycopg2.extensions.connection

    def __init__(self, conn):
        self.conn = conn

    @classmethod
    def connect(cls,
                host: str,
                port: int = 5432,
                user: str = 'postgres',
                password: str = 'password',
                dbname: str = 'postgres'):
        return cls(conn=psycopg2.connect(
            host=host, port=port, user=user, password=password, dbname=dbname)
        )

    def exec_query(self, query: str, params: List[str]) -> Generator:
        cursor = self.conn.cursor()
        try:
            cursor.execute(query, params)
        except psycopg2.Error as e:
            self.conn.rollback()
            raise e
        return cursor.fetchall()

    def exec(self, sql: str, params: List[str]):
        cursor = self.conn.cursor()
        try:
            cursor.execute(sql, params)
        except psycopg2.Error as e:
            self.conn.rollback()
            raise e
        self.conn.commit()

In [63]:
import os

PG_USER = os.environ.get('PG_USER', 'postgres')
PG_PASS = os.environ.get('PG_PASS', 'password')
PG_HOST = os.environ.get('PG_HOST', '172.17.0.2')
PG_PORT = os.environ.get('PG_PORT', 5432)
PG_DBNAME = os.environ.get('PG_DBNAME', 'vnkrtv')

In [66]:
storage = PostgresStorage.connect(
    host=PG_HOST,
    port=PG_PORT,
    user=PG_USER,
    password=PG_PASS,
    dbname=PG_DBNAME
)

In [88]:
from typing import Generator, Iterable
import re

import tqdm
import nltk
import razdel


class Tokenizer:
    remove_punctuation = re.compile(r'[^а-яА-ЯёЁ ]')

    @classmethod
    def tokenize(cls, text: str) -> Generator:
        return (cls.remove_punctuation.sub('', sentence.text.lower().strip())
                for sentence in razdel.sentenize(text))


class TextProcessor:
    tokenizer = Tokenizer()

    @classmethod
    def get_sentences_gens(cls, texts: Iterable) -> Generator:
        for text in texts:
            for sentence in cls.tokenizer.tokenize(text):
                yield sentence
                
    @classmethod
    def get_word_lists_gen(cls, texts: Iterable) -> Generator:
        for text in tqdm.tqdm(texts):
            for sentence in cls.tokenizer.tokenize(text):
                yield [_.text for _ in razdel.tokenize(sentence)]

    @classmethod
    def get_text_gen(cls, text_gen: Iterable) -> Generator:
        for sentence in cls.get_sentences_gens(text_gen):
            yield [_.text for _ in razdel.tokenize(sentence)]

    @classmethod
    def get_ngram_gen(cls, text_gen: Iterable, ngram_size: int = 3) -> Generator:
        for sentence in cls.get_sentences_gens(text_gen):
            yield [''.join(item) for item in nltk.ngrams(sentence, ngram_size)]


In [70]:
train_text = [
    row[0] for row in storage.exec_query('''
    SELECT text
      FROM posts INNER JOIN habs 
        ON posts.post_id = habs.post_id 
     WHERE hab = %s 
     LIMIT 2000''', params=['Машинное обучение'])
]
len(train_text)

2000

In [96]:
t9_index = {
  "settings": {
    "index": {
      "analysis": {
        "analyzer": {
          "custom_analyzer": {
            "type": "custom",
            "tokenizer": "standard",
            "filter": [
              "lowercase",
              "custom_edge_ngram"
            ]
          }
        },
        "filter": {
          "custom_edge_ngram": {
            "type": "edge_ngram",
            "min_gram": 2,
            "max_gram": 10
          }
        }
      }
    }
  },
  "mappings": {
    "sentence": {
      "properties": {
        "text": {
          "type": "text",
          "analyzer": "custom_analyzer",
          "search_analyzer": "standard"
        }
      }
    }
  }
}
es.indices.create('t9_app_3', body=t9_index)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 't9_app_3'}

In [98]:
window_size = 5

for sentence in TextProcessor.get_word_lists_gen(train_text):
#     for i in range(len(sentence) // window_size):
#         words = sentence[i * window_size: (i + 1) * window_size]
#         text = ' '.join(sentence[i * window_size: (i + 1) * window_size])
#         print(text)
    es.index(index='t9_app_3', doc_type='sentence', body={
        'text': ' '.join(sentence)
    })
#         if len(words) < window_size:
#             break






  0%|          | 0/2000 [00:00<?, ?it/s]




  0%|          | 1/2000 [00:02<1:21:08,  2.44s/it]




  0%|          | 2/2000 [00:03<1:04:12,  1.93s/it]




  0%|          | 3/2000 [00:03<51:44,  1.55s/it]  




  0%|          | 4/2000 [00:04<43:57,  1.32s/it]




  0%|          | 5/2000 [00:05<40:25,  1.22s/it]




  0%|          | 6/2000 [00:06<40:06,  1.21s/it]




  0%|          | 7/2000 [00:07<31:26,  1.06it/s]




  0%|          | 8/2000 [00:07<25:28,  1.30it/s]




  0%|          | 9/2000 [00:08<27:03,  1.23it/s]




  0%|          | 10/2000 [00:08<23:08,  1.43it/s]




  1%|          | 11/2000 [00:09<21:55,  1.51it/s]




  1%|          | 12/2000 [00:13<54:10,  1.63s/it]




  1%|          | 13/2000 [00:14<45:34,  1.38s/it]




  1%|          | 14/2000 [00:14<37:00,  1.12s/it]




  1%|          | 15/2000 [00:15<33:45,  1.02s/it]




  1%|          | 16/2000 [00:16<30:15,  1.09it/s]




  1%|          | 17/2000 [00:16<29:23,  1.12it/s]




  1%|          | 18/2000 [00:17<29:

 12%|█▏        | 237/2000 [05:25<36:33,  1.24s/it]




 12%|█▏        | 238/2000 [05:28<50:20,  1.71s/it]




 12%|█▏        | 239/2000 [05:30<54:14,  1.85s/it]




 12%|█▏        | 240/2000 [05:32<55:26,  1.89s/it]




 12%|█▏        | 241/2000 [05:34<55:56,  1.91s/it]




 12%|█▏        | 242/2000 [05:35<47:15,  1.61s/it]




 12%|█▏        | 243/2000 [05:36<42:59,  1.47s/it]




 12%|█▏        | 244/2000 [05:36<34:04,  1.16s/it]




 12%|█▏        | 245/2000 [05:37<27:07,  1.08it/s]




 12%|█▏        | 246/2000 [05:38<31:49,  1.09s/it]




 12%|█▏        | 247/2000 [05:39<30:04,  1.03s/it]




 12%|█▏        | 248/2000 [05:41<36:28,  1.25s/it]




 12%|█▏        | 249/2000 [05:41<27:11,  1.07it/s]




 12%|█▎        | 250/2000 [05:42<29:15,  1.00s/it]




 13%|█▎        | 251/2000 [05:43<31:57,  1.10s/it]




 13%|█▎        | 252/2000 [05:45<32:39,  1.12s/it]




 13%|█▎        | 253/2000 [05:45<29:27,  1.01s/it]




 13%|█▎        | 254/2000 [05:46<29:56,  1.03s/it]




 13%|█▎   

 24%|██▎       | 472/2000 [11:02<36:38,  1.44s/it]




 24%|██▎       | 473/2000 [11:04<35:40,  1.40s/it]




 24%|██▍       | 475/2000 [11:05<31:23,  1.24s/it]




 24%|██▍       | 476/2000 [11:08<41:46,  1.64s/it]




 24%|██▍       | 477/2000 [11:09<36:10,  1.42s/it]




 24%|██▍       | 478/2000 [11:11<44:16,  1.75s/it]




 24%|██▍       | 479/2000 [11:12<36:57,  1.46s/it]




 24%|██▍       | 480/2000 [11:13<35:40,  1.41s/it]




 24%|██▍       | 481/2000 [11:14<29:37,  1.17s/it]




 24%|██▍       | 482/2000 [11:22<1:18:31,  3.10s/it]




 24%|██▍       | 483/2000 [11:23<1:02:34,  2.47s/it]




 24%|██▍       | 484/2000 [11:25<1:03:55,  2.53s/it]




 24%|██▍       | 485/2000 [11:26<52:01,  2.06s/it]  




 24%|██▍       | 486/2000 [11:27<43:37,  1.73s/it]




 24%|██▍       | 487/2000 [11:27<33:04,  1.31s/it]




 24%|██▍       | 488/2000 [11:28<30:12,  1.20s/it]




 24%|██▍       | 489/2000 [11:30<30:53,  1.23s/it]




 24%|██▍       | 490/2000 [11:31<29:35,  1.18s/it]




 2

 35%|███▌      | 706/2000 [16:50<19:29,  1.11it/s]




 35%|███▌      | 707/2000 [16:50<15:01,  1.43it/s]




 35%|███▌      | 708/2000 [16:52<24:21,  1.13s/it]




 35%|███▌      | 709/2000 [16:55<32:35,  1.51s/it]




 36%|███▌      | 710/2000 [16:58<44:00,  2.05s/it]




 36%|███▌      | 711/2000 [17:00<39:56,  1.86s/it]




 36%|███▌      | 712/2000 [17:01<37:11,  1.73s/it]




 36%|███▌      | 713/2000 [17:02<29:45,  1.39s/it]




 36%|███▌      | 714/2000 [17:02<23:17,  1.09s/it]




 36%|███▌      | 715/2000 [17:02<18:18,  1.17it/s]




 36%|███▌      | 716/2000 [17:03<14:32,  1.47it/s]




 36%|███▌      | 717/2000 [17:04<17:32,  1.22it/s]




 36%|███▌      | 718/2000 [17:04<17:21,  1.23it/s]




 36%|███▌      | 719/2000 [17:05<13:28,  1.58it/s]




 36%|███▌      | 720/2000 [17:06<15:50,  1.35it/s]




 36%|███▌      | 722/2000 [17:07<14:32,  1.46it/s]




 36%|███▌      | 723/2000 [17:08<19:10,  1.11it/s]




 36%|███▌      | 724/2000 [17:10<23:36,  1.11s/it]




 36%|███▋ 

 47%|████▋     | 942/2000 [22:56<27:05,  1.54s/it]




 47%|████▋     | 943/2000 [22:59<32:38,  1.85s/it]




 47%|████▋     | 944/2000 [23:01<33:01,  1.88s/it]




 47%|████▋     | 945/2000 [23:01<26:11,  1.49s/it]




 47%|████▋     | 946/2000 [23:02<23:09,  1.32s/it]




 47%|████▋     | 947/2000 [23:03<19:43,  1.12s/it]




 47%|████▋     | 948/2000 [23:04<17:58,  1.03s/it]




 47%|████▋     | 949/2000 [23:05<16:42,  1.05it/s]




 48%|████▊     | 950/2000 [23:06<16:07,  1.09it/s]




 48%|████▊     | 951/2000 [23:16<1:03:52,  3.65s/it]




 48%|████▊     | 952/2000 [23:18<56:49,  3.25s/it]  




 48%|████▊     | 953/2000 [23:18<41:41,  2.39s/it]




 48%|████▊     | 954/2000 [23:18<29:57,  1.72s/it]




 48%|████▊     | 955/2000 [23:19<26:31,  1.52s/it]




 48%|████▊     | 956/2000 [23:20<23:10,  1.33s/it]




 48%|████▊     | 957/2000 [23:21<17:04,  1.02it/s]




 48%|████▊     | 958/2000 [23:21<16:21,  1.06it/s]




 48%|████▊     | 959/2000 [23:22<14:02,  1.24it/s]




 48%|█

 59%|█████▊    | 1173/2000 [28:34<18:51,  1.37s/it]




 59%|█████▊    | 1174/2000 [28:37<25:24,  1.85s/it]




 59%|█████▉    | 1175/2000 [28:38<21:16,  1.55s/it]




 59%|█████▉    | 1176/2000 [28:39<19:35,  1.43s/it]




 59%|█████▉    | 1177/2000 [28:40<19:14,  1.40s/it]




 59%|█████▉    | 1178/2000 [28:42<20:00,  1.46s/it]




 59%|█████▉    | 1179/2000 [28:42<15:10,  1.11s/it]




 59%|█████▉    | 1180/2000 [28:42<11:17,  1.21it/s]




 59%|█████▉    | 1181/2000 [28:44<16:44,  1.23s/it]




 59%|█████▉    | 1182/2000 [28:45<13:49,  1.01s/it]




 59%|█████▉    | 1183/2000 [28:46<13:54,  1.02s/it]




 59%|█████▉    | 1184/2000 [28:47<13:58,  1.03s/it]




 59%|█████▉    | 1185/2000 [28:56<46:45,  3.44s/it]




 59%|█████▉    | 1186/2000 [28:57<35:37,  2.63s/it]




 59%|█████▉    | 1187/2000 [28:59<33:38,  2.48s/it]




 59%|█████▉    | 1188/2000 [29:00<26:51,  1.98s/it]




 59%|█████▉    | 1189/2000 [29:00<19:58,  1.48s/it]




 60%|█████▉    | 1190/2000 [29:01<19:58,  1.48s/

 70%|███████   | 1403/2000 [34:15<13:13,  1.33s/it]




 70%|███████   | 1404/2000 [34:16<11:59,  1.21s/it]




 70%|███████   | 1405/2000 [34:16<08:56,  1.11it/s]




 70%|███████   | 1406/2000 [34:18<13:00,  1.31s/it]




 70%|███████   | 1407/2000 [34:19<13:11,  1.33s/it]




 70%|███████   | 1408/2000 [34:22<17:57,  1.82s/it]




 70%|███████   | 1409/2000 [34:24<15:41,  1.59s/it]




 70%|███████   | 1410/2000 [34:24<12:07,  1.23s/it]




 71%|███████   | 1411/2000 [34:25<12:22,  1.26s/it]




 71%|███████   | 1412/2000 [34:32<29:21,  3.00s/it]




 71%|███████   | 1413/2000 [34:33<21:18,  2.18s/it]




 71%|███████   | 1414/2000 [34:33<15:23,  1.58s/it]




 71%|███████   | 1415/2000 [34:34<13:31,  1.39s/it]




 71%|███████   | 1416/2000 [34:35<14:35,  1.50s/it]




 71%|███████   | 1417/2000 [34:36<10:43,  1.10s/it]




 71%|███████   | 1418/2000 [34:37<10:22,  1.07s/it]




 71%|███████   | 1419/2000 [34:38<10:45,  1.11s/it]




 71%|███████   | 1420/2000 [34:39<10:36,  1.10s/

 82%|████████▏ | 1635/2000 [39:45<08:23,  1.38s/it]




 82%|████████▏ | 1636/2000 [39:46<07:24,  1.22s/it]




 82%|████████▏ | 1637/2000 [39:47<07:01,  1.16s/it]




 82%|████████▏ | 1638/2000 [39:50<09:31,  1.58s/it]




 82%|████████▏ | 1639/2000 [39:50<07:11,  1.20s/it]




 82%|████████▏ | 1640/2000 [39:51<06:27,  1.08s/it]




 82%|████████▏ | 1641/2000 [39:51<05:58,  1.00it/s]




 82%|████████▏ | 1642/2000 [39:54<08:04,  1.35s/it]




 82%|████████▏ | 1643/2000 [39:55<08:11,  1.38s/it]




 82%|████████▏ | 1644/2000 [39:56<06:59,  1.18s/it]




 82%|████████▏ | 1645/2000 [39:56<05:25,  1.09it/s]




 82%|████████▏ | 1646/2000 [39:58<06:26,  1.09s/it]




 82%|████████▏ | 1647/2000 [39:59<06:17,  1.07s/it]




 82%|████████▏ | 1648/2000 [40:01<08:59,  1.53s/it]




 82%|████████▏ | 1649/2000 [40:02<07:40,  1.31s/it]




 82%|████████▎ | 1650/2000 [40:02<06:03,  1.04s/it]




 83%|████████▎ | 1651/2000 [40:08<13:48,  2.37s/it]




 83%|████████▎ | 1652/2000 [40:09<11:30,  1.98s/

 93%|█████████▎| 1867/2000 [45:10<02:18,  1.04s/it]




 93%|█████████▎| 1868/2000 [45:11<01:47,  1.22it/s]




 93%|█████████▎| 1869/2000 [45:11<01:43,  1.27it/s]




 94%|█████████▎| 1870/2000 [45:12<01:43,  1.26it/s]




 94%|█████████▎| 1871/2000 [45:16<03:46,  1.75s/it]




 94%|█████████▎| 1872/2000 [45:17<03:03,  1.43s/it]




 94%|█████████▎| 1873/2000 [45:19<03:22,  1.59s/it]




 94%|█████████▎| 1874/2000 [45:19<02:27,  1.17s/it]




 94%|█████████▍| 1875/2000 [45:20<02:01,  1.03it/s]




 94%|█████████▍| 1876/2000 [45:20<01:43,  1.20it/s]




 94%|█████████▍| 1877/2000 [45:20<01:23,  1.47it/s]




 94%|█████████▍| 1878/2000 [45:24<02:56,  1.45s/it]




 94%|█████████▍| 1879/2000 [45:24<02:14,  1.11s/it]




 94%|█████████▍| 1880/2000 [45:26<02:30,  1.25s/it]




 94%|█████████▍| 1881/2000 [45:27<02:40,  1.35s/it]




 94%|█████████▍| 1882/2000 [45:28<02:05,  1.06s/it]




 94%|█████████▍| 1883/2000 [45:30<02:54,  1.49s/it]




 94%|█████████▍| 1884/2000 [45:32<02:55,  1.51s/

In [74]:
import tqdm

for sentence in tqdm.tqdm(train_corpus):
    es.index(index='test_t9', doc_type='my_type', body={
        'text': sentence
    })

 17%|█▋        | 31900/187405 [08:04<42:24, 61.11it/s]  

KeyboardInterrupt: 

In [99]:
from typing import List, Dict, Any

from elasticsearch import Elasticsearch


class ElasticEngine:
    es: Elasticsearch
    doc_type: str = "sentence"

    def __init__(self, host: str, **kwargs):
        self.es = Elasticsearch(host=host, **kwargs)

    def add_index(self, name: str, number_of_shards: int = 1, number_of_replicas: int = 2):
        self.es.indices.create(index=name, body={
            "settings": {
                "index": {
                    "number_of_shards": number_of_shards,
                    "number_of_replicas": number_of_replicas,
                    "analysis": {
                        "analyzer": {
                            "t9_analyzer": {
                                "type": "custom",
                                "tokenizer": "standard",
                                "filter": [
                                    "lowercase",
                                    "custom_edge_ngram"
                                ]
                            }
                        },
                        "filter": {
                            "custom_edge_ngram": {
                                "type": "edge_ngram",
                                "min_gram": 2,
                                "max_gram": 10
                            }
                        }
                    }
                }
            },
            "mappings": {
                self.doc_type: {
                    "properties": {
                        "text": {
                            "type": "text",
                            "analyzer": "t9_analyzer",
                            "search_analyzer": "standard"
                        }
                    }
                }
            }
        })

    def add_doc(self, index_name: str, text: str):
        self.es.index(index=index_name, doc_type=self.doc_type, body={
            'text': text
        })

    def get(self, index_name: str, phrase: str, count: int = 10) -> List[str]:
        phrase_len = len(phrase)
        sentences = [
            doc['_source']['text']
            for doc in self.es.search(
                index=index_name,
                doc_type=self.doc_type,
                body={
                    "query": {
                        "match_phrase": {
                            "text": phrase
                        }
                    },
                    "size": count
                })['hits']['hits']
        ]
        return [
            sentence[sentence.find(phrase) + phrase_len + 1:]
            for sentence in sentences
        ]

In [100]:
es = ElasticEngine(host='localhost')

In [107]:
phrase = input('Input: ')
s = es.get(index_name='t9_app_3', phrase=phrase, count=5)
s

Input: нейронная


['визуализациястандартная нейронная сетьсвёрточная нейронная сетьс левой стороны стандартная х слойная нейронная сеть',
 'обучение модели сводится к поочередному обучению дискриминатора и генератора в цикле до сходимости ниже приведены графики для четырех моделей дискриминаторатрехслойная нейронная сетьтрехслойная нейронная сеть с регуляризациейтрехслойная нейронная сеть с регуляризациейтрехслойная нейронная сеть с и регуляризацией',
 'свёрточная нейронная сеть снс или рекуррентная нейронная сеть рнс обрабатывающая порядок слов',
 'рекуррентная нейронная сетьрекуррентная нейронная сеть рнс это тип инс который хорошо подходит для решения задач связанных с временными рядами',
 'нейронная сеть с пятью слоями это просто нейронная сеть с четырьмя слоями передающая данные в какието перцептроны']

In [111]:
s[0][s[0].find(phrase) + len(phrase) + 1:]

'сетьсвёрточная нейронная сетьс левой стороны стандартная х слойная нейронная сеть'

In [15]:
!{sys.executable} -m pip install elasticsearch>=7

In [14]:
import elasticsearch
elasticsearch.__version__

(6, 8, 2)